# Read the npz file and convert to txt file
# Then extract four momentum of the leading 100 particles within the jet

In [13]:
import pandas as pd
import numpy as np
import math

data_with_labels = np.load('/home/daohan/apps/qgtagging/QG_jets_1.npz')
data = data_with_labels['X']
labels = data_with_labels['y']

sorted_data = np.zeros_like(data)
for i in range(data.shape[0]):
    sorted_data[i] = data[i][np.argsort(-data[i,:,0])]
datax = sorted_data[:,0:100,:]

n_points = datax.shape[1]

# Save the data to a text file
with open('/home/daohan/apps/qgtagging/train_data1.txt', 'w') as f:
    for i, point_cloud in enumerate(datax):
        for point in point_cloud:
            row_str = ' '.join([str(x) for x in point])
            f.write(row_str)
            f.write('\n')
        if (i+1) % n_points == 0:  # Add a "#" after every point cloud
            f.write('#')
        if (i+1) != data.shape[0]*n_points:
            f.write('\n') # Add a newline after every point except for the last one
print(datax.shape)

with open("/home/daohan/apps/qgtagging/train_data1.txt", 'r') as oldf, open('/home/daohan/apps/qgtagging/train_data1y.txt', 'w') as newf:
    lines = oldf.readlines()
    for i, line in enumerate(lines, start=1):
        ls = line.split()
        if len(ls) == 4:
            if all(float(val) != 0 for val in ls[:2]):
                new_line = "   ".join(ls[:4]) + "   \n"
                newf.write(new_line)
        if i % 101 == 0 or len(ls) == 1:
            newf.write("#\n")
            
label_file_path = "/home/daohan/apps/qgtagging/train_label1.txt"
with open(label_file_path, 'w') as f_label:
    for lbl in labels:
        f_label.write(str(lbl) + "\n")

(100000, 100, 4)


# Normalize the PID information and extract (E,px,py,pz,pT,eta,phi,PID) information

In [4]:
import math

with open("/home/daohan/apps/qgtagging/train_data1y.txt", 'r') as oldf, \
     open('/home/daohan/apps/qgtagging/train_data1z.txt', 'w') as newf:

    pid_map = {
        22: 0,    211: 0.1,  -211: 0.2,  321: 0.3,  -321: 0.4,
        130: 0.5, 2112: 0.6, -2112: 0.7, 2212: 0.8, -2212: 0.9,
        11: 1,    -11: 1.1,  13: 1.2,    -13: 1.3
    }

    for line in oldf:
        ls = line.split()

        if len(ls) == 4:
            pt, eta, phi, PID = map(float, ls)
            if phi > math.pi:
                phi -= math.pi

            px = pt * math.cos(phi)
            py = pt * math.sin(phi)
            pz = pt * math.sinh(eta)
            E  = pt * math.cosh(eta)

            if PID in pid_map:
                PID = pid_map[PID]

            newf.write(f"{E} {px} {py} {pz} {pt} {eta} {phi} {PID}\n")

        elif len(ls) == 1:  
            newf.write("#\n")

# Preparation of the particle input features 

In [6]:
import math

with open("/home/daohan/apps/qgtagging/train_data1z.txt",'r') as oldf:
    lines = oldf.readlines()

PTJ, ETAJ, PHIJ, EJ = [], [], [], []

px_sum = py_sum = pz_sum = E_sum = 0.0

for line in lines:
    ls = line.split()
    if len(ls) == 8:
        px_sum += float(ls[1])
        py_sum += float(ls[2])
        pz_sum += float(ls[3])
        E_sum  += float(ls[0])

    elif len(ls) == 1:
        pt_sum  = (px_sum**2 + py_sum**2)**0.5
        eta_sum = -math.log(pt_sum / (pz_sum + (pt_sum**2 + pz_sum**2)**0.5))
        phi_sum = math.acos(px_sum / pt_sum)

        PTJ.append(pt_sum)
        ETAJ.append(eta_sum)
        PHIJ.append(phi_sum)
        EJ.append(E_sum)

        px_sum = py_sum = pz_sum = E_sum = 0.0

decimal = 5
i = 0 

with open('/home/daohan/apps/qgtagging/train_data1a.txt','w') as newf:
    for line in lines:
        ls = line.split()
        if len(ls) == 8:
            E   = float(ls[0])
            px  = float(ls[1])
            py  = float(ls[2])
            pz  = float(ls[3])
            pt  = float(ls[4])
            eta = float(ls[5])
            phi = float(ls[6])
            PID = float(ls[7])

            if eta > 3.14159:
                eta -= 3.14159

            px_r  = round(px, decimal)
            py_r  = round(py, decimal)
            pz_r  = round(pz, decimal)
            E_r   = round(E,  decimal)
            pt_r  = round(pt, decimal)
            eta_r = round(eta, decimal)
            phi_r = round(phi, decimal)

            ptc       = round(pt_r / PTJ[i], decimal)
            Ec        = round(E_r  / EJ[i],  decimal)
            delta_eta = round(eta_r - ETAJ[i], decimal)
            delta_phi = round(phi_r - PHIJ[i], decimal)
            delta_R   = round((delta_eta**2 + delta_phi**2)**0.5, decimal)

            newf.write(f"{E_r}   {px_r}   {py_r}   {pz_r}   {pt_r}   {eta_r}   {phi_r}   "
                       f"{ptc}   {Ec}   {delta_eta}   {delta_phi}   {delta_R}   {PID}   \n")

        elif len(ls) == 1:
            i += 1
            newf.write("#\n")


# Preparation of the jet input features 

In [10]:
import math
import numpy as np

def init_accumulator():
    return {
        'px': 0.0,
        'py': 0.0,
        'pz': 0.0,
        'E': 0.0,
        'detasum': 0.0,
        'dphisum': 0.0,
        'drsum': 0.0
    }

pids_of_interest = [0.0, 0.1, 0.2, 0.3, 0.4,
                    0.5, 0.6, 0.7, 0.8, 0.9]

def init_all_accumulators():
    accumulators = {}
    accumulators['total'] = init_accumulator()
    for pid_key in pids_of_interest:
        accumulators[pid_key] = init_accumulator()
    return accumulators

acc = init_all_accumulators()

PTJ = []
EJ = []
PXJ = []
PYJ = []
PZJ = []
ETAJ = []
PHIJ = []
RJ   = [] 

EJPID  = []
PXJPID = []
PYJPID = []
PZJPID = []
PTJPID = []
PTFPID = []
EFPID  = []
ETAPID = []
PHIPID = []
RPID   = []

def finalize_event(accumulators):

    total = accumulators['total']
    pxsum = total['px']
    pysum = total['py']
    pzsum = total['pz']
    Esum  = total['E']

    pt_sum = math.sqrt(pxsum**2 + pysum**2) if pxsum or pysum else 0.0
    Detasum = total['detasum'] / pt_sum if pt_sum else 0.0
    Dphisum = total['dphisum'] / pt_sum if pt_sum else 0.0
    Drsum   = total['drsum']   / pt_sum if pt_sum else 0.0

    pid_pt_sums = {}
    pid_E_sums  = {}
    pid_deta    = {}
    pid_dphi    = {}
    pid_dr      = {}

    for pid_key in pids_of_interest:
        px_ = accumulators[pid_key]['px']
        py_ = accumulators[pid_key]['py']
        E_  = accumulators[pid_key]['E']
        pt_ = math.sqrt(px_**2 + py_**2) if px_ or py_ else 0.0

        # 加权平均
        d_eta = accumulators[pid_key]['detasum'] / pt_ if pt_ else 0.0
        d_phi = accumulators[pid_key]['dphisum'] / pt_ if pt_ else 0.0
        d_r   = accumulators[pid_key]['drsum']   / pt_ if pt_ else 0.0

        pid_pt_sums[pid_key] = pt_
        pid_E_sums[pid_key]  = E_
        pid_deta[pid_key]    = d_eta
        pid_dphi[pid_key]    = d_phi
        pid_dr[pid_key]      = d_r

    PTJ_S = [pid_pt_sums[p] for p in pids_of_interest] 
    EJ_S  = [pid_E_sums[p]  for p in pids_of_interest]     
    PTF_S = [pt / pt_sum if pt_sum else 0.0 for pt in PTJ_S]
    EF_S  = [e  / Esum   if Esum  else 0.0 for e  in EJ_S ]

    max_pt = max(PTJ_S) if PTJ_S else 0
    idx_max_pt = PTJ_S.index(max_pt) if PTJ_S else -1
    chosen_pid = pids_of_interest[idx_max_pt] if idx_max_pt >= 0 else None

    DetasumPID = pid_deta[chosen_pid]  if chosen_pid is not None else 0.0
    DphisumPID = pid_dphi[chosen_pid]  if chosen_pid is not None else 0.0
    DrsumPID   = pid_dr[chosen_pid]    if chosen_pid is not None else 0.0

    PTJ.append(pt_sum)
    EJ.append(Esum)
    PXJ.append(pxsum)
    PYJ.append(pysum)
    PZJ.append(pzsum)
    ETAJ.append(Detasum)
    PHIJ.append(Dphisum)
    RJ.append(Drsum)

    EJPID.append(max(EJ_S) if EJ_S else 0.0)
    PXJPID.append(max(abs(accumulators[p]['px']) for p in pids_of_interest))
    PYJPID.append(max(abs(accumulators[p]['py']) for p in pids_of_interest))
    PZJPID.append(max(abs(accumulators[p]['pz']) for p in pids_of_interest))
    PTJPID.append(max_pt)
    PTFPID.append(max(PTF_S) if PTF_S else 0.0)
    EFPID.append(max(EF_S)   if EF_S  else 0.0)
    ETAPID.append(DetasumPID)
    PHIPID.append(DphisumPID)
    RPID.append(DrsumPID)

    return init_all_accumulators()

with open("/home/daohan/apps/qgtagging/train_data1a.txt", 'r') as oldf:
    for line in oldf:
        ls = line.split()
        if len(ls) == 13:
            E   = float(ls[0])
            px  = float(ls[1])
            py  = float(ls[2])
            pz  = float(ls[3])
            pt  = float(ls[4])
            dEta = float(ls[9])
            dPhi = float(ls[10])
            dR   = float(ls[11])
            PID  = float(ls[12])
            acc['total']['px']      += px
            acc['total']['py']      += py
            acc['total']['pz']      += pz
            acc['total']['E']       += E
            acc['total']['detasum'] += dEta * pt
            acc['total']['dphisum'] += dPhi * pt
            acc['total']['drsum']   += dR   * pt

            if PID in acc:
                acc[PID]['px']      += px
                acc[PID]['py']      += py
                acc[PID]['pz']      += pz
                acc[PID]['E']       += E
                acc[PID]['detasum'] += dEta * pt
                acc[PID]['dphisum'] += dPhi * pt
                acc[PID]['drsum']   += dR   * pt

        elif len(ls) == 1:
            acc = finalize_event(acc)

EJ    = np.array(EJ)
PXJ   = np.array(PXJ)
PYJ   = np.array(PYJ)
PZJ   = np.array(PZJ)
PTJ   = np.array(PTJ)
PTFJ  = np.ones(100000)
EFJ   = np.ones(100000)

ETAJ  = np.array(ETAJ)
PHIJ  = np.array(PHIJ)
RJ    = np.array(RJ)

EJPID  = np.array(EJPID)
PXJPID = np.array(PXJPID)
PYJPID = np.array(PYJPID)
PZJPID = np.array(PZJPID)
PTJPID = np.array(PTJPID)
PTFPID = np.array(PTFPID)
EFPID  = np.array(EFPID)
ETAPID = np.array(ETAPID)
PHIPID = np.array(PHIPID)
RPID   = np.array(RPID)

print(EJ[5]) if len(EJ) > 5 else None

final = np.column_stack((
    EJ, PXJ, PYJ, PZJ, PTJ,
    PTFJ, EFJ, ETAJ, PHIJ, RJ,
    EJPID, PXJPID, PYJPID, PZJPID,
    PTJPID, PTFPID, EFPID, ETAPID, PHIPID, RPID
))
print(final.shape)
np.save('/home/daohan/apps/qgtagging/train_jet_information1.npy', final)


579.7002299999999
(100000, 20)
[ 9.16403880e+02  4.79956650e+02  1.06384820e+02  7.67024780e+02
  4.91605651e+02  1.00000000e+00  1.00000000e+00 -1.69712004e-02
  2.30796776e-02  6.54640336e-02  2.03250970e+02  1.07786510e+02
  2.32954300e+01  1.70707890e+02  1.10275150e+02  2.24316279e-01
  2.21791913e-01 -6.93120923e-03 -5.25772704e-03  1.65294968e-02]


# Convert to numpy format

In [4]:
import numpy as np
import os

data_file_path = "/home/daohan/apps/qgtagging/train_data1a.txt"
label_file_path = "/home/daohan/apps/qgtagging/train_label1.txt"
save_path1 = "/home/daohan/apps/qgtagging/train_data_with_labels1_original.npz"
save_path2 = "/home/daohan/apps/qgtagging/train_data_with_labels1.npz"

data = []
with open(data_file_path, 'r') as f:
    points = []
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line == "#":
            data.append(np.array(points, dtype=float))
            points = []
        else:
            row = list(map(float, line.split()))
            points.append(row)
    if points:
        data.append(np.array(points, dtype=float))

point_counts = [len(evt) for evt in data]

with open(label_file_path, 'r') as f:
    labels = [float(line.strip()) for line in f]
labels = np.array(labels, dtype=float)

max_point_count = 100
feature_dim = data[0].shape[1] if data else 0
data_flat = np.zeros((len(data), max_point_count, feature_dim), dtype=float)

for i, evt_array in enumerate(data):
    count_i = len(evt_array)
    data_flat[i, : min(count_i, max_point_count), :] = evt_array[:max_point_count]

data_reduced = np.delete(data_flat, [1,2,3], axis=2)
np.savez(save_path1, data=data_flat, labels=labels)
np.savez(save_path2, data=data_reduced, labels=labels)


print("data shape:", data_flat.shape)
print("labels shape:", labels.shape)
print("data shape:", data_reduced.shape)
print("labels shape:", labels.shape)




data shape: (100000, 100, 13)
labels shape: (100000,)
data shape: (100000, 100, 10)
labels shape: (100000,)


# Split

In [23]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

data = np.load('/home/daohan/apps/qgtagging/train_data_with_labels1.npz')
data_with_labels1 = data['data'][:10000]
labels1 = data['labels'][:10000]
data_with_labels2 = data['data'][10000:20000]
labels2 = data['labels'][10000:20000]
data_with_labels3 = data['data'][20000:30000]
labels3 = data['labels'][20000:30000]
data_with_labels4 = data['data'][30000:40000]
labels4 = data['labels'][30000:40000]
data_with_labels5 = data['data'][40000:50000]
labels5 = data['labels'][40000:50000]
data_with_labels6 = data['data'][50000:60000]
labels6 = data['labels'][50000:60000]
data_with_labels7 = data['data'][60000:70000]
labels7 = data['labels'][60000:70000]
data_with_labels8 = data['data'][70000:80000]
labels8 = data['labels'][70000:80000]
data_with_labels9 = data['data'][80000:90000]
labels9 = data['labels'][80000:90000]
data_with_labels10 = data['data'][90000:100000]
labels10 = data['labels'][90000:100000]
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_1.npz',data=data_with_labels1, labels=labels1)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_2.npz',data=data_with_labels2, labels=labels2)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_3.npz',data=data_with_labels3, labels=labels3)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_4.npz',data=data_with_labels4, labels=labels4)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_5.npz',data=data_with_labels5, labels=labels5)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_6.npz',data=data_with_labels6, labels=labels6)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_7.npz',data=data_with_labels7, labels=labels7)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_8.npz',data=data_with_labels8, labels=labels8)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_9.npz',data=data_with_labels9, labels=labels9)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_10.npz',data=data_with_labels10, labels=labels10)

In [5]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

data = np.load('/home/daohan/apps/qgtagging/train_data_with_labels1_original.npz')
data_with_labels1 = data['data'][:10000]
labels1 = data['labels'][:10000]
data_with_labels2 = data['data'][10000:20000]
labels2 = data['labels'][10000:20000]
data_with_labels3 = data['data'][20000:30000]
labels3 = data['labels'][20000:30000]
data_with_labels4 = data['data'][30000:40000]
labels4 = data['labels'][30000:40000]
data_with_labels5 = data['data'][40000:50000]
labels5 = data['labels'][40000:50000]
data_with_labels6 = data['data'][50000:60000]
labels6 = data['labels'][50000:60000]
data_with_labels7 = data['data'][60000:70000]
labels7 = data['labels'][60000:70000]
data_with_labels8 = data['data'][70000:80000]
labels8 = data['labels'][70000:80000]
data_with_labels9 = data['data'][80000:90000]
labels9 = data['labels'][80000:90000]
data_with_labels10 = data['data'][90000:100000]
labels10 = data['labels'][90000:100000]
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_1_original.npz',data=data_with_labels1, labels=labels1)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_2_original.npz',data=data_with_labels2, labels=labels2)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_3_original.npz',data=data_with_labels3, labels=labels3)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_4_original.npz',data=data_with_labels4, labels=labels4)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_5_original.npz',data=data_with_labels5, labels=labels5)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_6_original.npz',data=data_with_labels6, labels=labels6)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_7_original.npz',data=data_with_labels7, labels=labels7)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_8_original.npz',data=data_with_labels8, labels=labels8)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_9_original.npz',data=data_with_labels9, labels=labels9)
np.savez('/home/daohan/apps/qgtagging/train_data_with_labels1_10_original.npz',data=data_with_labels10, labels=labels10)

# KNN

In [2]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

infile = "/home/daohan/apps/qgtagging/train_data_with_labels1_1.npz"
data = np.load(infile)
data_with_labels = data['data']       # shape: (10000, 100, 10)
labels = data['labels']

print("Original shape:", data_with_labels.shape)  # (10000, 100, 10)

data_with_labels[..., 0] = np.log(data_with_labels[..., 0] + 1e-8)  # log(E)
data_with_labels[..., 1] = np.log(data_with_labels[..., 1] + 1e-8)  # log(pt)

data_with_neighbors = np.zeros((10000, 100, 21, 8), dtype=float)

num_events = data_with_labels.shape[0]
num_points = data_with_labels.shape[1]

knn_neighbors = 21

for i in range(num_events):
    curr_data = data_with_labels[i]

    eta_coords = curr_data[:, 2]
    phi_coords = curr_data[:, 3]

    coords = np.stack([eta_coords, phi_coords], axis=-1)
   
    knn = NearestNeighbors(n_neighbors=knn_neighbors)
    knn.fit(coords)

    distances, indices = knn.kneighbors(coords)

    for j in range(num_points):
        for k in range(knn_neighbors):
            neighbor_idx = indices[j, k]

            logE_neighbor  = curr_data[neighbor_idx, 0]  # log(E)
            logPt_neighbor = curr_data[neighbor_idx, 1]  # log(pt)
            ptc_neighbor   = curr_data[neighbor_idx, 4]
            Ec_neighbor    = curr_data[neighbor_idx, 5]
            PID_neighbor   = curr_data[neighbor_idx, 9]

            eta_diff = curr_data[neighbor_idx, 2] - curr_data[j, 2]
            phi_diff = curr_data[neighbor_idx, 3] - curr_data[j, 3]
            dR = np.sqrt(eta_diff**2 + phi_diff**2)

            data_with_neighbors[i, j, k, 0] = logE_neighbor
            data_with_neighbors[i, j, k, 1] = logPt_neighbor
            data_with_neighbors[i, j, k, 2] = ptc_neighbor
            data_with_neighbors[i, j, k, 3] = Ec_neighbor
            data_with_neighbors[i, j, k, 4] = eta_diff
            data_with_neighbors[i, j, k, 5] = phi_diff
            data_with_neighbors[i, j, k, 6] = dR
            data_with_neighbors[i, j, k, 7] = PID_neighbor

outfile = "/home/daohan/apps/qgtagging/train_data_with_neighbors1.npz"
np.savez(outfile, data=data_with_neighbors, labels=labels)

print("Final shape:", data_with_neighbors.shape)  # (10000, 100, 21, 8)



Original shape: (10000, 100, 10)
Final shape: (10000, 100, 5, 8)


# Calculate the particle interaction matrix

In [1]:
import numpy as np

# 1) 加载数据 (10000, 100, 13)
in_path  = '/home/daohan/apps/qgtagging/train_data_with_labels1_1_original.npz'
out_path = '/home/daohan/apps/qgtagging/train_interact1.npz'

data_dict = np.load(in_path)
data_all  = data_dict['data']  # 形状: (10000, 100, 13)
print("Loaded shape:", data_all.shape)

# 只取前 10000 个事件 (实际上正好是 10000)
point_cloud = data_all[:10000]  
N, n_points = point_cloud.shape[0], point_cloud.shape[1]  # (10000, 100)
print("Using N =", N, "events, each with", n_points, "particles.")

# 2) 基于列 5 (eta) 和列 6 (phi) 计算两两差分 => deltaR
eta_i = point_cloud[:, :, 5]  # eta
phi_i = point_cloud[:, :, 6]  # phi

delta_eta = eta_i[:, :, np.newaxis] - eta_i[:, np.newaxis, :]
delta_phi = phi_i[:, :, np.newaxis] - phi_i[:, np.newaxis, :]
deltaR = np.sqrt(delta_eta**2 + delta_phi**2)  # (10000, 100, 100)

# 3) 计算 pairwise mass (log| ...)
#    基于列 0,1,2,3 => (E, px, py, pz)
E_col  = np.exp(point_cloud[:, :, 0])  # e^E
px_col = np.exp(point_cloud[:, :, 1])  # e^px
py_col = np.exp(point_cloud[:, :, 2])  # e^py
pz_col = np.exp(point_cloud[:, :, 3])  # e^pz

E_sum  = E_col[:, :, np.newaxis]  + E_col[:,  np.newaxis, :]
px_sum = px_col[:, :, np.newaxis] + px_col[:, np.newaxis, :]
py_sum = py_col[:, :, np.newaxis] + py_col[:, np.newaxis, :]
pz_sum = pz_col[:, :, np.newaxis] + pz_col[:, np.newaxis, :]

mass_raw = (E_sum**2 - px_sum**2 - py_sum**2 - pz_sum**2)
mass = np.log(np.abs(mass_raw) + 1e-12)

# 4) 计算 kT = log| deltaR * exp(min(pt_i, pt_j)) |
pt_vals = point_cloud[:, :, 4]  # 第4列: pt (实际存的可能是log(pt) 或类似?)
pt_min = np.exp(
    np.minimum(pt_vals[:, :, np.newaxis],
               pt_vals[:, np.newaxis, :])
)
kt_raw = deltaR * pt_min
kt     = np.log(np.abs(kt_raw) + 1e-12)

# 5) 计算 z = exp(min(pt_i, pt_j)) / ( exp(pt_i) + exp(pt_j) )
pt_i = np.exp(pt_vals)[:, :, np.newaxis]
pt_j = np.exp(pt_vals)[:, np.newaxis, :]
z = pt_min / (pt_i + pt_j + 1e-12)

# 6) 计算 ptd = log| exp(pt_i) - exp(pt_j) |
pt_diff = pt_i - pt_j
ptd     = np.log(np.abs(pt_diff) + 1e-12)

# 7) 合并以上 5 个特征 => (N, 100, 100, 5)
feat_stack = np.stack([deltaR, mass, kt, z, ptd], axis=-1)

# 8) 增加一个空维 (for PID mask) => (N, 100, 100, 6)
new_feature = np.concatenate(
    [feat_stack, np.zeros((N, n_points, n_points, 1), dtype=feat_stack.dtype)],
    axis=-1
)

# 9) 使用 第12列 PID，相等时在最后一维存放mask
PID_col = point_cloud[:, :, 12]  # shape: (N,100)
for i in range(n_points):      
    for j in range(n_points):  
        mask = (PID_col[:, i] == PID_col[:, j])  # (N,)
        new_feature[:, i, j, 5] = mask.astype(feat_stack.dtype)

# 10) 对角线置为1
for i in range(n_points):
    new_feature[:, i, i, :] = 1

# 检查 NaN
has_nan = np.isnan(new_feature).any()
if has_nan:
    print("Warning: new_feature contains NaN values")

print("Final shape:", new_feature.shape)

# 11) 保存为 npz 文件
np.savez(out_path, data=new_feature)
print(f"Saved shape {new_feature.shape} to {out_path}")



Loaded shape: (10000, 100, 13)
Using N = 100 events, each with 100 particles.
Final shape: (100, 100, 100, 6)
Saved shape (100, 100, 100, 6) to /home/daohan/apps/qgtagging/train_interact1.npz


/tmp/ipykernel_96731/1791711266.py:36: RuntimeWarning: overflow encountered in square
  mass_raw = (E_sum**2 - px_sum**2 - py_sum**2 - pz_sum**2)


# Calculate the jet interaction matrix

In [4]:
import numpy as np
a = np.load('/home/daohan/apps/qgtagging/train_jet_information1.npy')
b = np.zeros((100000,11,11))
for i in range(11):
    b[:,i,i] = 1
for i in range(7,10):
    for j in range(0,5):
        b[:,i,j] = 0
for i in range(0,5):
    for j in range(7,10):
        b[:,i,j] = 0
#first column        
b[:,0,1] = a[:,1]/a[:,0]
b[:,0,2] = a[:,2]/a[:,0]
b[:,0,3] = a[:,3]/a[:,0]
b[:,0,4] = a[:,4]/a[:,0]
b[:,0,6] = 1
b[:,0,10] = np.log(a[:,10])
#second column
b[:,1,0] = a[:,1]/a[:,0]
b[:,1,4] = a[:,1]/a[:,4]
b[:,1,10] = np.log(a[:,11])
#third column
b[:,2,0] = a[:,2]/a[:,0]
b[:,2,4] = a[:,2]/a[:,4]
b[:,2,10] = np.log(a[:,12])
#fourth column
b[:,3,0] = a[:,3]/a[:,0]
b[:,3,10] = np.log(a[:,13])
#fifth column
b[:,4,0] = a[:,4]/a[:,0]
b[:,4,1] = a[:,1]/a[:,4]
b[:,4,2] = a[:,2]/a[:,4]
b[:,4,5] = 1
b[:,4,10] = np.log(a[:,14])
#sixth column
b[:,5,4] = 1
b[:,5,10] = a[:,15]
#seventh column
b[:,6,0] = 1
b[:,6,10] = a[:,16]


#eighth column
b[:,7,9] = a[:,7]/a[:,9]
b[:,7,10] = a[:,17]
#nineth column
b[:,8,9] = a[:,8]/a[:,9]
b[:,8,10] = a[:,18]
#tenth column
b[:,9,7] = a[:,7]/a[:,9]
b[:,9,8] = a[:,8]/a[:,9]
b[:,9,10] = a[:,19]
#eleventh column
b[:,10,0] = np.log(a[:,10])
b[:,10,1] = np.log(a[:,11])
b[:,10,2] = np.log(a[:,12])
b[:,10,3] = np.log(a[:,13])
b[:,10,4] = np.log(a[:,14])
b[:,10,5] = a[:,15]
b[:,10,6] = a[:,16]
b[:,10,7] = a[:,17]
b[:,10,8] = a[:,18]
b[:,10,9] = a[:,19]



print(b.shape)
np.save('/home/daohan/apps/qgtagging/train_interact_1.npy', b)


(100000, 11, 11)
